<a href="https://colab.research.google.com/github/rdemarqui/topic_modeling/blob/main/02.%20topic_modeling_customer_claim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Customer claims analysis

## Open Dataset

In [1]:
from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore', 'error')

In [2]:
import pandas as pd

In [3]:
data_url = 'https://raw.githubusercontent.com/rdemarqui/topic_modeling/main/datasets/full_dataset_claro.xlsx'
df = pd.read_excel(data_url)
print(df.shape)
df.head()

(7000, 4)


,title,link,problem,description
0,Cobrança indevida,/claro/cobranca-indevida_T1CYGOYSZNTs1n-3/,Cobrança indevida,Recebi um e-mail spam com uma fatura no valor ...
1,Cobrança indevida,/claro/cobranca-indevida_SjlfHnzsQ6W7KVee/,Cobrança indevida,"Compra não autoriza na meu cartão, chip de núm..."
2,Recarga por aplicativo da claro,/claro/recarga-por-aplicativo-da-claro_8WMAcH4...,Cobrança indevida,No dia 11 de outubro realizei uma recarga pelo...
3,Cobrança duplicada,/claro/cobranca-duplicada__97YKYI-f-o1AkRd/,Cobrança indevida,segue o número de protocolo do atendimento rea...
4,EMPRESA CLARO MOVEL NÃO ESTÁ CREDITANDO RECARG...,/claro/empresa-claro-movel-nao-esta-creditando...,Cobrança indevida,FIZ HOJE DIA 12/10/2023 UMA RECARGA DE CELULAR...


In [4]:
# Check category quantity
df.groupby('problem')['description'].count()

problem
Cancelamento                               500
Cobrança indevida                          500
Consumo de crédito                         500
Corte indevido da linha                    500
Demora na execução                         500
Instabilidade do sinal                     500
Mau atendimento do prestador de serviço    500
Mudança de plano                           498
Planos e tarifas                           500
Portabilidade                              500
Problema com recarga                       500
Problemas com o chip                       500
Promoções                                  500
Qualidade da internet                      500
Name: description, dtype: int64

## Text Preprocess

The dataset that we will use was acquired through web scrapping on reclameaqui.com.br. This data is messy, since the users can write everithing they want. Because of that, we'll need to clean and make some modifications bofore we create our model.

### Normalize

Firstly, we will normalize our text. This proccess goes through some steps such as removing punctuation, accentuation, special characters and masked words.

In [5]:
import string

In [6]:
punctuations = list(string.punctuation)

def remove_punctuation(text):
  return ''.join([char if char not in punctuations else ' ' for char in text])

In [7]:
accentuation = {
    "á": "a", "ã": "a", "à": "a","â": "a",
    "é": "e","ê": "e",
    "í": "i",
    "ó": "o","õ": "o", "ô":"o",
    "ú": "u",
    "ç": "c"
    }

def remove_accentuation(text):
  return ''.join(accentuation.get(char, char) for char in text)

In [8]:
special_charaters = [
    "1","2","3","4","5","6","7","8","9","0",
    " a "," b "," c "," d "," e "," f "," g "," h "," i "," j "," k "," l ",
    " m "," n "," o "," p "," q "," r "," s "," t "," u "," v "," x "," z ",
    "r$", "$",
    ]

def remove_special_characters(text):
  return ''.join([char if char not in special_charaters else ' ' for char in text])

Reclame Aqui website often masks data when it is sensitive information or offensive words using "editado por reclame aqui" in place. We will remove it from our database.

In [9]:
def remove_mask(text):
  return text.replace("editado pelo reclame aqui", "mascara")

In [10]:
def normalize_text(df, text_field, lower=True, rem_punct=True, rem_accent=True, rem_spec_caract=True, rem_mask=True):
  df[text_field + "_clean"] = df[text_field].astype(str)
  if lower: df[text_field + "_clean"] = df[text_field + "_clean"].str.lower()
  if rem_punct: df[text_field + "_clean"] = df[text_field + "_clean"].apply(remove_punctuation)
  if rem_accent: df[text_field + "_clean"] = df[text_field + "_clean"].apply(remove_accentuation)
  if rem_spec_caract: df[text_field + "_clean"] = df[text_field + "_clean"].apply(remove_special_characters)
  if rem_mask: df[text_field + "_clean"] = df[text_field + "_clean"].apply(remove_mask)
  df[text_field + "_clean"] = df[text_field + "_clean"].replace(r'\s+', ' ', regex=True) #remove spaces

In [11]:
# Normalizing text in a new column
normalize_text(df, 'description')

### Stop Words

Stop words make the topics poor. Removing stop words can reduce noise and improve the efficiency of text analysis algorithms.

In [12]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
stops_nltk = nltk.corpus.stopwords.words('portuguese')
stop_words = list(stops_nltk)
stop_words = [remove_accentuation(word) for word in stop_words]
stop_words = list(set(stop_words))

def remove_stop_words(text):
    words_list = text.split()
    words_list = [word for word in words_list if word not in stop_words]
    text = ' '.join(words_list)

    return text

In [14]:
# Removing Stop Words
df["description_clean_stop"] = df["description_clean"].apply(remove_stop_words)

### Lemmatize

Lemmatization is a natural language processing technique that reduces words to their base or root form. It helps in standardizing words, improving text analysis, and simplifying language understanding.

In [15]:
import spacy.cli

language_models = ['pt_core_news_sm', 'pt_core_news_md', 'pt_core_news_lg']
spacy_model = language_models[2]

disable = ['tagger', 'parser', 'ner', 'entity_ruler', 'entity_linker', 'textcat',
           'morphologizer','attribute_ruler']

try:
  nlp = spacy.load(spacy_model, disable=disable)
except:
  spacy.cli.download(spacy_model)
  nlp = spacy.load(spacy_model, disable=disable)

In [16]:
def lemma_pipe(doc_list):
  lemma_text_list = []
  for doc in nlp.pipe(doc_list, n_process=-1):
      lemma_text_list.append(" ".join(token.lemma_ for token in doc))

  return lemma_text_list

In [17]:
# Applying lemmatization
df["description_clean_stop_lemma"] = lemma_pipe(df["description_clean_stop"])

### TF-IDF

Removing low TF-IDF words is essential in topic modeling because these words probably provide little discriminatory power and can dominate topics. By excluding them, the model focuses on more meaningful and distinctive terms, improving topic quality and helping to identify relevant keywords that truly characterize the underlying topics in a corpus.

In [18]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

import gensim
import gensim.corpora as corpora
from gensim.models import TfidfModel

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [19]:
documents = df["description_clean_stop_lemma"]

In [20]:
# Tokenize text
token_docs = [word_tokenize(text) for text in documents]

# Convert text to frequency dictionary
dictionary = corpora.Dictionary(token_docs)

# Convert text in bag of words
corpus = [dictionary.doc2bow(text) for text in token_docs]

# Calculating TF-IDF
tfidf = TfidfModel(corpus, id2word=dictionary)

In [21]:
# Filtering low TF-IDF values
low_value = 0.01
low_value_words = []
for bow in corpus:
    low_value_words += [id for id, value in tfidf[bow] if value < low_value]

# Removing low TF-IDF from dictionary
dictionary.filter_tokens(bad_ids=low_value_words)

# Recompute the corpus with low value words are filtered out:
corpus = [dictionary.doc2bow(doc) for doc in token_docs]

## Latent Dirichlet Allocation (LDA)

As we have 14 claim categories, we will use this value as a parameter in num of topics.

In [22]:
# LDA implmentation
lda_model = gensim.models.LdaModel(corpus,
                                   num_topics=14,
                                   id2word=dictionary,
                                   random_state=42,
                                   update_every=1,
                                   chunksize=100,
                                   passes=15,
                                   alpha="auto")

### Visualization

To visualize the topics created by our model, we will use pyLDAvis.

Documentation: https://pypi.org/project/pyLDAvis/

In [44]:
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [43]:
warnings.filterwarnings("ignore", category=FutureWarning)

In [24]:
try:
  import pyLDAvis.gensim
except:
  !pip install pyLDAvis==2.1.2
  clear_output()
  import pyLDAvis.gensim

In [42]:
vis = pyLDAvis.gensim.prepare(lda_model, corpus, lda_model.id2word, mds="mmds", R=20)
pyLDAvis.display(vis)

## BERTopic

As we could see above, work with LDA demands a lot of text prepocessing. Now we will use a completely different alternative called Bertopic.

Documentation: https://maartengr.github.io/BERTopic/index.html

In [45]:
device = !nvidia-smi -L
device = "cuda" if device[0][:3] == 'GPU' else "cpu"
device

'cuda'

In [ ]:
try:
  from bertopic import BERTopic
except:
  !pip install bertopic
  clear_output()
  from bertopic import BERTopic

In [ ]:
try:
  from sentence_transformers import SentenceTransformer
except:
  !pip install sentence_transformers
  clear_output()
  from sentence_transformers import SentenceTransformer

In [29]:
sentence_model = SentenceTransformer('rufimelo/bert-large-portuguese-cased-sts',
                                     device=device)
clear_output()

In [ ]:
docs = df.description_clean_stop
classes = df.problem

In [ ]:
# Prepare embeddings
embeddings = sentence_model.encode(docs, show_progress_bar=False)

In [ ]:
# Train BERTopic
topic_model = BERTopic(embedding_model=sentence_model, nr_topics=14)
topics, probs = topic_model.fit_transform(docs, embeddings)

In [46]:
# Visualize
topic_model.visualize_topics()

In [34]:
topic_model.visualize_barchart(top_n_topics=15)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [47]:
topic_model.visualize_heatmap()

In [48]:
topics_per_class = topic_model.topics_per_class(docs, classes=classes)
topic_model.visualize_topics_per_class(topics_per_class)

In [49]:
topic_model.visualize_hierarchy()

In [50]:
# Run the visualization with the original embeddings
topic_model.visualize_documents(docs, embeddings=embeddings)